In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/266_project/generated_non_microaggressive_equivalents_train.csv")
validation_df  = pd.read_csv("/content/drive/MyDrive/266_project/generated_non_microaggressive_equivalents_validation.csv")
test_df   = pd.read_csv("/content/drive/MyDrive/266_project/generated_non_microaggressive_equivalents_test.csv")


In [ ]:
train_df["split"] = "train"
test_df["split"]  = "test"
validation_df["split"]   = "val"


In [ ]:
#combine into one dataset, csv format

combined_df_generated_micro = pd.concat([train_df, test_df, validation_df], ignore_index=True)


In [ ]:
#sanity check

combined_df_generated_micro.head()


,original_text,original_label,generated_non_microaggressive_equivalent,human_annotation,split
0,If I can't do the accent how can I say it the ...,1,I want to say it respectfully and accurately. ...,NaN,train
1,I'm not a misogynist! I don't hate woman! It's...,1,Women and men are equally capable and intellig...,NaN,train
2,Oh... So I'm just going to put that you aren't...,1,I'll note that you're not currently sexually a...,NaN,train
3,"My friend Kelly; she's black, but she's really...",1,My friend Kelly is really smart.,NaN,train
4,"God, they should just shut up and be happy for...",1,I wish they would share their feelings without...,NaN,train


In [ ]:
#save on drive

combined_df_generated_micro.to_csv("/content/drive/MyDrive/266_project/combined_df_generated_micro.csv", index=False)


In [ ]:
#split into two datasets so we can test reading level of each

df_original_text = combined_df_generated_micro[["original_text"]].copy()

df_generated_text = combined_df_generated_micro[["generated_non_microaggressive_equivalent"]].copy()

In [ ]:
#sanity check
df_original_text.head()

,original_text
0,If I can't do the accent how can I say it the ...
1,I'm not a misogynist! I don't hate woman! It's...
2,Oh... So I'm just going to put that you aren't...
3,"My friend Kelly; she's black, but she's really..."
4,"God, they should just shut up and be happy for..."


In [ ]:
#sanity check
df_generated_text.head()

,generated_non_microaggressive_equivalent
0,I want to say it respectfully and accurately. ...
1,Women and men are equally capable and intellig...
2,I'll note that you're not currently sexually a...
3,My friend Kelly is really smart.
4,I wish they would share their feelings without...


In [ ]:
#installs for readability tests

!pip install -q textstat tqdm

import numpy as np
from tqdm import tqdm
import textstat
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.9 MB/s eta 0:00:00


In [ ]:


def safe_metric(func, text):
    try:
        return func(text)
    except Exception:
        return np.nan

def readability_metrics(text: str):
    text = str(text).strip()
    return {
        "flesch_reading_ease":         safe_metric(textstat.flesch_reading_ease, text),
        "flesch_kincaid_grade":        safe_metric(textstat.flesch_kincaid_grade, text),
        "gunning_fog":                 safe_metric(textstat.gunning_fog, text),
        "smog_index":                  safe_metric(textstat.smog_index, text),
        "coleman_liau_index":          safe_metric(textstat.coleman_liau_index, text),
        "automated_readability_index": safe_metric(textstat.automated_readability_index, text),
        "dale_chall_readability":      safe_metric(textstat.dale_chall_readability_score, text),
        "linsear_write":               safe_metric(textstat.linsear_write_formula, text),
        "difficult_words":             safe_metric(textstat.difficult_words, text),
        "avg_sentence_length":         safe_metric(textstat.avg_sentence_length, text),
        "avg_syllables_per_word":      safe_metric(textstat.avg_syllables_per_word, text),
        "text_standard_grade":         safe_metric(lambda t: textstat.text_standard(t, float_output=False), text),
        "word_count":                  len(text.split()) if text else 0,
        "char_count":                  len(text),
    }

def compute_readability_df(df_in: pd.DataFrame, text_col="text") -> pd.DataFrame:
    rows = []
    for t in tqdm(df_in[text_col].tolist(), desc=f"Computing readability for '{text_col}'"):
        rows.append(readability_metrics(t))
    return pd.DataFrame(rows)


# Run on the two datasets


read_orig = compute_readability_df(df_original_text, text_col="original_text")
read_gen  = compute_readability_df(df_generated_text, text_col="generated_non_microaggressive_equivalent")

# Attach metrics back to the original rows (suffix to avoid name collisions)
df_orig_metrics = pd.concat([df_original_text.reset_index(drop=True), read_orig.add_suffix("_orig")], axis=1)
df_gen_metrics  = pd.concat([df_generated_text.reset_index(drop=True),  read_gen.add_suffix("_gen")],  axis=1)

# =========================
# Summary comparison
# =========================
NUMERIC_COLS = [
    "flesch_reading_ease",
    "flesch_kincaid_grade",
    "gunning_fog",
    "smog_index",
    "coleman_liau_index",
    "automated_readability_index",
    "dale_chall_readability",
    "linsear_write",
    "difficult_words",
    "avg_sentence_length",
    "avg_syllables_per_word",
    "word_count",
    "char_count",
]

summary = pd.DataFrame({
    "metric": NUMERIC_COLS,
    "Original_mean":  [read_orig[c].mean() for c in NUMERIC_COLS],
    "Generated_mean": [read_gen[c].mean()  for c in NUMERIC_COLS],
})
summary["Delta(Gen-Orig)"] = summary["Generated_mean"] - summary["Original_mean"]

print("\n=== Readability Comparison: Original vs Generated ===")
print(summary)



Computing readability for 'original_text':   0%|          | 0/1300 [00:00<?, ?it/s]/tmp/ipython-input-3579526066.py:3: DeprecationWarning: The 'avg_sentence_length' method has been deprecated due to being the same as 'words_per_sentence'. This method will be removed in thefuture.
  return func(text)
Computing readability for 'original_text': 100%|██████████| 1300/1300 [00:01<00:00, 826.95it/s] 
Computing readability for 'generated_non_microaggressive_equivalent': 100%|██████████| 1300/1300 [00:00<00:00, 5272.76it/s]


=== Readability Comparison: Original vs Generated ===
                         metric  Original_mean  Generated_mean  \
0           flesch_reading_ease      85.381980       72.475537   
1          flesch_kincaid_grade       3.751233        5.544465   
2                   gunning_fog       6.065988        7.485714   
3                    smog_index       6.312119        7.769928   
4            coleman_liau_index       4.094519        6.246904   
5   automated_readability_index       4.284074        6.020216   
6        dale_chall_readability       6.465403        7.039882   
7                 linsear_write       4.537381        5.142231   
8               difficult_words       1.852308        2.173077   
9           avg_sentence_length       9.664685        9.636679   
10       avg_syllables_per_word       1.319662        1.472556   
11                   word_count      15.467692       11.022308   
12                   char_count      82.409231       63.056923   

    Delta(Gen-Orig) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1. Choose numeric readability feature columns
# -----------------------------

NUMERIC_COLS = [
    "flesch_reading_ease",
    "flesch_kincaid_grade",
    "gunning_fog",
    "smog_index",
    "coleman_liau_index",
    "automated_readability_index",
    "dale_chall_readability",
    "linsear_write",
    "difficult_words",
    "avg_sentence_length",
    "avg_syllables_per_word",
    "word_count",
    "char_count",
]

# -----------------------------
# 2. Build feature matrix X and labels y
# -----------------------------

# Label originals as 0
X_orig = read_orig[NUMERIC_COLS].copy()
y_orig = pd.Series(0, index=X_orig.index)

# Label generated texts as 1
X_gen = read_gen[NUMERIC_COLS].copy()
y_gen = pd.Series(1, index=X_gen.index)

# Combine into one dataset
X = pd.concat([X_orig, X_gen], ignore_index=True)
y = pd.concat([y_orig, y_gen], ignore_index=True)

# Optional: inspect
print("Shape of combined feature matrix:", X.shape)
print("Class balance:\n", y.value_counts())

# -----------------------------
# 3. Train/test split
# -----------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# -----------------------------
# 4. Standardize features (helps logistic regression)
# -----------------------------

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# -----------------------------
# 5. Fit trivial classifier (Logistic Regression)
# -----------------------------

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_scaled, y_train)

# -----------------------------
# 6. Evaluate model
# -----------------------------

y_pred = clf.predict(X_test_scaled)
y_proba = clf.predict_proba(X_test_scaled)[:, 1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print("\n=== READABILITY-ONLY CLASSIFIER PERFORMANCE ===")
print(f"Accuracy:   {acc:.4f}")
print(f"ROC-AUC:    {auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Original", "Generated"]))

# -----------------------------
# 7. Optional: inspect feature weights
# -----------------------------

coef_df = pd.DataFrame({
    "feature": NUMERIC_COLS,
    "weight": clf.coef_[0]
}).sort_values("weight", ascending=False)

print("\n=== Feature Influence (Positive → predicts 'Generated') ===")
print(coef_df)


Shape of combined feature matrix: (2600, 13)
Class balance:
 0    1300
1    1300
Name: count, dtype: int64

=== READABILITY-ONLY CLASSIFIER PERFORMANCE ===
Accuracy:   0.7038
ROC-AUC:    0.7863

Classification Report:
              precision    recall  f1-score   support

    Original       0.74      0.64      0.68       390
   Generated       0.68      0.77      0.72       390

    accuracy                           0.70       780
   macro avg       0.71      0.70      0.70       780
weighted avg       0.71      0.70      0.70       780


=== Feature Influence (Positive → predicts 'Generated') ===
                        feature    weight
7                 linsear_write  1.429074
8               difficult_words  1.043072
4            coleman_liau_index  0.334295
10       avg_syllables_per_word  0.239258
1          flesch_kincaid_grade -0.005894
3                    smog_index -0.059132
0           flesch_reading_ease -0.145191
5   automated_readability_index -0.257596
6        dale_ch